In [ ]:
#Preamble and packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
from functools import partial
print("Script is running, please be patient")

In [ ]:
# Collecting links to each player's page
# Page has link to each position-page
positionRoot = "https://overthecap.com/contract-history/"
positionStemVect = ["quarterback","running-back", "fullback","wide-receiver","tight-end","left-tackle","left-guard","center","right-guard","right-tackle","interior-defensive-line","edge-rusher","linebacker","safety","cornerback","kicker","punter","long-snapper"]

In [ ]:
# Initialize empty pandas dataframe, to which each a df of players (by position) will be concatenated
playerPagedf = pd.DataFrame(columns=['name', 'link', 'position'])

In [ ]:

#Scrape each position page for a dataframe of players.
for stem in positionStemVect:
    print("attempting: "+ stem)
     # Create unique URL for each position page
    url = positionRoot + stem
    
    # load page html
    page = requests.get(url)
    # Parse html to be readable - This script uses the html.parser parser which is slower than lxml's parser but does not require additional dependencies or installs
    soup = BeautifulSoup(page.content, 'html.parser')
    # The "table" has the "position-table" CSS class
    table = soup.find(class_='position-table')

    #initialize vectors for each variable of interest
    player_names = []
    player_page_links = []
    player_position = []

    # Identify the non-header rows of the table
    tableBody = table.find('tbody')
    tableRows = tableBody.find_all('tr')

    # For each non-header row, collect the link and name
    for row in tableRows:
        playerObject = row.find('td')
        #get the 'href' attribute from the first 'a' tagged element
        playerLinkStem = playerObject.find('a').get('href')
        playerFullURL = "https://overthecap.com" + playerLinkStem
        #get the string contents of the first column of the given row
        playerName = playerObject.string

        # Insert the name, link, and position into the vectors of interest
        player_names.append(playerName)
        player_page_links.append(playerFullURL)
        player_position.append(stem)

    # Format the above vectors as a temporary pandas dataframe
    tempPositiondf = pd.DataFrame({
        'name' : player_names,
        'link' : player_page_links,
        'position' : player_position
    })

    # Row-bind temporary (position) df to master (all-players) df
    playerPagedf = pd.concat([playerPagedf, tempPositiondf], ignore_index=True)
    print("completed: " + stem)

#Initailize a long, player-year-contract-level dataframe
playerContractdf = pd.DataFrame()

#Initailize a vector to store broken pages for future checking
breakPages = []

In [ ]:
#Scraping each player's contract history data
playerPagedf = playerPagedf.drop_duplicates()

#Drop some dead pages
#playerPagedf = playerPagedf[playerpagedf.name != ""]
playerPagedf = playerPagedf[playerpagedf.name != "Manase Hungalu"]
playerPagedf = playerPagedf[playerPagedf.name != "Cyrus Jones"]

#These are names/observations that have already been stored because they were scraped on previous runs of this block
def extract(list):
    return [item[0] for item in list]

already_names = list(playerPagedf['name'])
breaked_names = list(set(extract(breakPages)))
skip_names = already_names + breaked_names


#Begin looping over pages to scrape
nrows = playerPagedf.shape[0]
i=1
for index, player_row in playerPagedf.iterrows():
    print("name: " + player_row['name'] + " " + str(i) + "/" + str(nrows) )

    if player_row['name'] in skip_names:
        print("already scraped " + player_row['name'] + ": " + str(i) + "/" + str(nrows))
        i = i +1
        continue
    else:

        # Create unique URL for each position page
        url = player_row['link']
        
        # load page html
        page = requests.get(url)
        # Parse html to be readable - This script uses the html.parser parser which is slower than lxml's parser but does not require additional dependencies or installs
        soup = BeautifulSoup(page.content, 'html.parser')

        #Go to the contract history tab section
        history_tab = soup.find(id='contract-history')
        history_table = history_tab.find(class_='salary-cap-history')
        #Identify what kind of columns are present/absent by looking into the header
        
        try:
            history_table_header = history_table.find('thead').find('tr').find_all('th')
        except AttributeError:
            breakPages.append([player_row['name'], player_row['link']])
            print("broken/NA pages: " + str(len(breakPages)))
            i = i +1
            continue 
        
        
        #initialize vector for column titles
        temp_header_titles = []
        for col in history_table_header:
            #To return the only vector in a vect of vectors list then take the first index
            header_col = col.attrs
            header_col_values = header_col.values()
            
            #Treat column headers with no attributes differently ("Team" col)
            if len(header_col_values) == 0:
                header_col_attrs = list(header_col_values)
            else:
                header_col_attrs = list(header_col_values)[0]
            
            #Omit the spacer columns
            if 'spacer' in header_col_attrs:
                pass
            else:
                header_col_title = col.string
                temp_header_titles.append(header_col_title)

        #initalize vector for actual contract data (each year is a row) so that we produce a vector of vectors (that is, a vector of individual years for each player)
        player_year_values = []
        history_table_rows = history_table.find('tbody').find_all('tr')

        #going year-by-year
        for row in history_table_rows:
            #initialize year vector
            year_values = []

            #find each cell for each row
            cols = row.find_all('td')
            #length adjustment
            for col in cols:
                attrs = col.attrs
                if len(attrs.values()) == 0:
                    col_attrs = list(attrs.values())
                else:
                    col_attrs = list(attrs.values())[0]
                
                # Omit spacer cells
                if 'spacer' in col_attrs:
                    pass
                else: 
                    col_value = col.string
                    #append value to the year vector
                    year_values.append(col_value)

            player_year_values.append(year_values)
        
        #store the header titles (column names) and values (immediately aboce) in a df
        playerdf = pd.DataFrame(
            player_year_values,
            columns=temp_header_titles
        )
        #Adding names and positions back to the player-specific df
        playerdf['name'] = player_row['name']
        playerdf['position'] = player_row['position']

        #row bind the player-specific df to the full df (playerContractdf)
        #pandas takes care of differences in columns by merging on the column name (same by virtue of scrape) and including new columns (reassigning missing values to 'NaN')
        playerContractdf = pd.concat([playerContractdf, playerdf], ignore_index=True)  
        i = i +1

In [ ]:
#cleaning up scraped data for export and processing
playerContractdf.columns = playerContractdf.columns.fillna('cap_number')
playerContractdf = playerContractdf.rename(columns={"Year" : "year",
    "Team" : "team",
    "Base Salary" : "base_salary",
    "Prorated Bonus" : "prorated_bonus",
    "Roster Bonus" : "roster_bonus",
    "Workout Bonus" : "workout_bonus",
    "Guaranteed Salary" : "guaranteed_salary",
    "Cap %" : "cap_percent",
    "Cash Paid" : 'cash_paid',
    "Other Bonus" : "other_bonus",
    "Per Game Roster Bonus" : "per_game_ros_bonus",
    "name" : "name",
    'position' : 'position' 
})

vars = ['base_salary', 'prorated_bonus', 'roster_bonus', 'workout_bonus', 'guaranteed_salary', 'cap_number', 'cap_percent', 'cash_paid', 'other_bonus', 'per_game_ros_bonus']
playerContractdf[vars] = playerContractdf[vars].replace({'\$' : '', ',' : '', '\%': ''}, regex=True)



playerContractdf.to_csv(r'total_export.csv', header=True, index=False)
